# Capacity Prediction

## Notebook 1: Basic Dataset Extraction

This notebook is dedicated to gathering all relevant information from spatial and operational sectors and saving it as a CSV file for further analysis.

**Author:** ACG  
**Date:** `2024-06`  

---

### Table of Contents
1. [Spatial sectors](#sectors-spatial)
2. [Operational sectors](#sectors-operational)
3. [Merge data](#merge)
4. [Saving as CSV](#save)

In [1]:
import sys
sys.path.append("..")
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from datetime import date

### DB connection and configurations <a id="configuration"></a>

In [2]:
from database.data_gatherer import (get_airspace_structures_spatial_for_all_sectors,
                                               get_airspace_structures_operational_all_sectors,
                                               get_spatial_airblocks_specific_atc_unit_code,
                                               get_all_cfmu_clustered_routes,
                                               get_all_interacting_cfmu_clustered_routes,
                                               get_oc_ec_groupby_date_traffic_volume)
from database.airspace_structures_operational import (compact_operational_by_capacity_date_from_date_to_for_specific_sector)

In [3]:
from utils.properties import *
from database.repository import (select_sectors_spatial,
                                            select_sectors_operational)
from utils.dwh_connection import get_dwh_connection

In [4]:
from utils.mining_utils import (merge_operational_capacity_for_specific_sector_to_spatial_structures,
                                lowest_highest_bound_specific_sector,
                                check_if_volume_with_steps,
                                volume_airblocks_composing_sector_average,
                                features_cfmu_clustered_routes_specific_sector,
                                features_cfmu_clustered_routes_interacting_spacific_sector)

from utils.time_processing import get_observations_for_specific_year

In [5]:
connection_dwh = get_dwh_connection(HOST_PERSEO_PRO, DWH_DATABASE, UID, PWD_PRO)

# Import spatial sectors datasets <a id="sectors-spatial"></a>

In [6]:
df_sectors_spatial = select_sectors_spatial(connection_dwh)
df_sectors_spatial.head(8)

,level_type,atcunit_code,atc_type,sector_code,date_from,date_to,active,sector_area,sector_perimeter,num_vertices,centroid_lat,centroid_lon
0,Sector,LECL,Aproximación,LECLDEW,2013-01-01,2013-04-04,False,2.953965e+10,9.861575e+05,29,37.440002,-0.412222
1,Sector,LECL,Aproximación,LECLAAW,2013-01-01,2015-05-28,False,9.628992e+09,3.810064e+05,10,38.000000,0.166667
2,Sector,LECL,Aproximación,LECLDEE,2013-01-01,2013-04-04,False,2.953969e+10,9.861545e+05,28,37.440002,-0.412222
3,Sector,LECL,Aproximación,LECLAAE,2013-01-01,2014-12-11,False,6.489498e+09,3.210091e+05,9,38.282223,-0.558056
4,Sector,LECL,Aproximación,LECLALL,2013-01-01,2013-04-04,False,5.495011e+10,1.070938e+06,32,37.488056,-0.353889
5,Sector,LECL,Aproximación,LECLSUP,2013-01-01,2013-04-04,False,5.495004e+10,1.070938e+06,31,37.440002,-0.412222
6,Sector,LECL,Aproximación,LECLTMS,2013-01-01,2013-04-04,False,5.495011e+10,1.070938e+06,32,37.488056,-0.353889
7,Sector,LECL,Aproximación,LECLVAP,2013-01-01,2013-04-04,False,2.009818e+10,5.890888e+05,14,39.250000,0.483333


### Corrección fechas

En la base de datos (SQLServer) los sectores que siguen abiertos tienen como fecha **date_to -> '9999-12-31'**, como la fecha máxima que se puede usar con los datetime64 es 2262-04-11 hay que corregirla para evitar este error: **OutOfBoundsDatetime**

In [7]:
print(df_sectors_spatial['date_to'].max())
print(pd.Timestamp.max)

9999-12-31
2262-04-11 23:47:16.854775807


In [8]:
df_sectors_spatial['date_to'] = pd.to_datetime(df_sectors_spatial['date_to'], errors='coerce').fillna(pd.Timestamp.max) #('2262-04-11 23:47:16.854775807')

# Operational sectors dataset <a id="sectors-operational"></a>

In [9]:
df_sectors_operational = select_sectors_operational(connection_dwh)
df_sectors_operational.head(8)

,level_type,atcunit_code,atc_type,sector_code,capacity,date_from,date_to,active
0,Sector,LECL,Aproximación,LECLALL,22,2013-01-01,2018-05-24,False
1,Sector,LECL,Aproximación,LECLVAP,26,2013-01-01,2013-04-04,False
2,Sector,LECL,Aproximación,LECLTMS,25,2013-01-01,2013-04-04,False
3,Sector,LECL,Aproximación,LECLTMW,36,2013-01-01,2022-03-24,False
4,Sector,LECL,Aproximación,LECLAAW,24,2013-01-01,2019-03-28,False
5,Sector,LECL,Aproximación,LECLDEW,40,2013-01-01,2013-04-04,False
6,Sector,LECL,Aproximación,LECLVAP,26,2013-01-01,2019-03-28,False
7,Sector,LECL,Aproximación,LECLDEE,40,2013-01-01,2013-04-04,False


In [10]:
df_sectors_operational['date_to'].max()

datetime.date(9999, 12, 31)

In [11]:
df_sectors_operational['date_to'] = pd.to_datetime(df_sectors_operational['date_to'], errors='coerce').fillna(pd.Timestamp.max)

In [12]:
# gets a list of sectors and atc units
atc_unit_codes = np.unique(df_sectors_spatial['atcunit_code'].values)
sector_codes = np.unique(df_sectors_spatial['sector_code'].values)

# Merge spatial and operational information with fluxes <a id="merge"></a>

In [13]:
df_all_sectors = pd.DataFrame()

In [14]:
# Information about all the clusteres routes
df_all_cfmu_clustered_routes = get_all_cfmu_clustered_routes(connection_dwh, year=2022)

# Information about all the interacting fluxes
df_all_cfmu_clustered_routes_interacting = get_all_interacting_cfmu_clustered_routes(connection_dwh, year=2022)

In [15]:
for atc_unit in tqdm(atc_unit_codes):    
    
    df_spatial_airblocks_specific_act_unit_code = get_spatial_airblocks_specific_atc_unit_code(connection_dwh, atc_unit)
    
    for sector in sector_codes:        
        
        df_spatial_sector = df_sectors_spatial[(df_sectors_spatial['atcunit_code'] == atc_unit) &
                                                   (df_sectors_spatial['sector_code'] == sector)]
        
        df_operational_sector = df_sectors_operational[(df_sectors_operational['atcunit_code'] == atc_unit) &
                                                           (df_sectors_operational['sector_code'] == sector)]
        df_spatial_sector['date_to'] = df_spatial_sector['date_to'].dt.date
        
        if len(df_spatial_sector) == 0 or len(df_operational_sector) == 0:
            continue
        else:            
            # shrinks the df so to have 1 capacity per valid period
            df_operational_sector_compacted = compact_operational_by_capacity_date_from_date_to_for_specific_sector(
                df_operational_sector)
            
            df_operational_sector_compacted['date_to'] = df_operational_sector_compacted['date_to'].dt.date
            
            # Merge spatial polygons with operational capacity
            df_dataset_specific_sector = merge_operational_capacity_for_specific_sector_to_spatial_structures(
                df_spatial_sector, df_operational_sector_compacted, debug_mode=False)
            
            # Add the lowest and highest bound of airblocks used to create the sector
            df_spatial_airblocks_specific_sector = df_spatial_airblocks_specific_act_unit_code[
                df_spatial_airblocks_specific_act_unit_code['sector_code'] == sector]
            
            df_dataset_specific_sector['lowest_bound'], df_dataset_specific_sector['highest_bound'] =  \
                zip(*df_dataset_specific_sector.apply(lambda x: lowest_highest_bound_specific_sector(
                    x.date_from, x.date_to, df_spatial_airblocks_specific_sector, debug_mode=False), axis=1))
            
            # + Check if sector touches the ground, the highest possible FL, or goes from ground to ceiling
            df_dataset_specific_sector['ground'] = df_dataset_specific_sector['lowest_bound'].apply(
                lambda lowestBound: 1 if lowestBound == 0 else 0)
            
            df_dataset_specific_sector['ceiling'] = df_dataset_specific_sector['highest_bound'].apply(
                lambda highestBound: 1 if highestBound == 660 else 0)
            
            df_dataset_specific_sector['from_ground_to_ceiling'] = \
                df_dataset_specific_sector[['lowest_bound', 'highest_bound']].apply(
                    lambda obs: 1 if (obs['lowest_bound'] == 0 and obs['highest_bound'] == 660) else 0, axis=1)
            
            # + Check if the volume has steps
            df_dataset_specific_sector['volume_with_steps'] = df_dataset_specific_sector.apply(
                lambda x: check_if_volume_with_steps(x.date_from, x.date_to, df_spatial_airblocks_specific_sector,
                                                     debug_mode=False), axis=1)
            
            # Add the volume of the sector based on lowest and highest bounds
            df_dataset_specific_sector.loc[:, 'volume_lowest_highest'] = df_dataset_specific_sector.loc[:, 'sector_area'] * \
                (df_dataset_specific_sector.loc[:, 'highest_bound'] - df_dataset_specific_sector.loc[:, 'lowest_bound'])
            
            # + From m3 to km3
            df_dataset_specific_sector.loc[:, 'volume_lowest_highest'] = \
                df_dataset_specific_sector.loc[:, 'volume_lowest_highest'] * 10e-9
            
            # Add the volume of the sector based on the available airblocks for the preriod
            df_dataset_specific_sector['volume_airblocks_avg'], df_dataset_specific_sector['num_airblocks_avg'] =  \
                zip(*df_dataset_specific_sector.apply(lambda x: volume_airblocks_composing_sector_average(
                    x.date_from, x.date_to, df_spatial_airblocks_specific_sector, debug_mode=False), axis=1))
            
            # + From m3 to km3
            df_dataset_specific_sector['volume_airblocks_avg'] = \
                df_dataset_specific_sector['volume_airblocks_avg'] * 10e-9
            
            # NOTE: Only samples from 2022 (fluxes)
            df_dataset_specific_sector = get_observations_for_specific_year(df_dataset_specific_sector, 
                                                                            year=2022, from_year_to_active=True)
            
            # Only procede if we have sample for the requested year
            if len(df_dataset_specific_sector) != 0:
                
                # Add features about the clustered routes for the sector
                df_cfmu_clustered_routes_specific_sector = df_all_cfmu_clustered_routes[
                    df_all_cfmu_clustered_routes['SectorCode'] == sector]

                df_dataset_specific_sector = features_cfmu_clustered_routes_specific_sector(
                    df_dataset_specific_sector, df_cfmu_clustered_routes_specific_sector)
            
                # Add features about the interacting fluxes
                df_cfmu_clustered_routes_interacting_specific_sector = df_all_cfmu_clustered_routes_interacting[
                    df_all_cfmu_clustered_routes_interacting['SectorCode'] == sector]
                list_all_clustered_routes_specific_sector = list(df_cfmu_clustered_routes_specific_sector['clusteredRouteKey'].values)
                
                df_dataset_specific_sector = features_cfmu_clustered_routes_interacting_spacific_sector(
                    df_dataset_specific_sector, df_cfmu_clustered_routes_interacting_specific_sector,
                    list_all_clustered_routes_specific_sector)
                
                # Add infor specific sector to the final dataset
                df_all_sectors = pd.concat([df_all_sectors, df_dataset_specific_sector], 
                                                    ignore_index=True)
            

100%|██████████| 9/9 [02:37<00:00, 17.51s/it]


In [16]:
pd.set_option('display.max_columns', None)
df_all_sectors.head(3)

,level_type,atcunit_code,sector_code,atc_type,sector_area,sector_perimeter,num_vertices,centroid_lat,centroid_lon,capacity,date_from,date_to,lowest_bound,highest_bound,ground,ceiling,from_ground_to_ceiling,volume_with_steps,volume_lowest_highest,volume_airblocks_avg,num_airblocks_avg,Num_fluxes,all_attitudesIN,all_attitudesOUT,attitude_cruise_cruise,attitude_cruise_descend_comb,attitude_cruise_climb_comb,attitude_decent_descend,attitude_climb_climb,attitude_descend_climb_comb,sum_nFlights,max_nFlights,avg_nFlights,median_nFlights,std_nFlights,percentile25_nFlights,percentile75_nFlights,sum_avgTimeInRoute,max_avgTimeInRoute,median_avgTimeInRoute,std_avgTimeInRoute,sum_avgTimeInRoute_divided_num_flight,percentile25_avgTimeInRoute,percentile75_avgTimeInRoute,all_latIN_fluxes,all_lngIN_fluxes,all_latOUT_fluxes,all_lngOUT_fluxes,avg_latIN_fluxes,avg_lngIN_fluxes,avg_latOUT_fluxes,avg_lngOUT_fluxes,num_interacting_fluxes,num_interacting_fluxes_type_0,num_interacting_fluxes_type_1,num_interacting_fluxes_type_2,num_interacting_fluxes_type_3,avg_interacting_type,median_interacting_type,max_num_flights_interacting_fluxes_type_1,avg_num_flights_interacting_fluxes_type_1,median_num_flights_interacting_fluxes_type_1,std_num_flights_interacting_fluxes_type_1,percentile25_num_flights_interacting_fluxes_type_1,percentile75_num_flights_interacting_fluxes_type_1,max_num_flights_interacting_fluxes_type_2,avg_num_flights_interacting_fluxes_type_2,median_num_flights_interacting_fluxes_type_2,std_num_flights_interacting_fluxes_type_2,percentile25_num_flights_interacting_fluxes_type_2,percentile75_num_flights_interacting_fluxes_type_2,max_num_flights_interacting_fluxes_type_3,avg_num_flights_interacting_fluxes_type_3,median_num_flights_interacting_fluxes_type_3,std_num_flights_interacting_fluxes_type_3,percentile25_num_flights_interacting_fluxes_type_3,percentile75_num_flights_interacting_fluxes_type_3,max_avg_time_route_type_1_combined,sum_avg_time_route_type_1_divided_num_flight_combined,median_avg_time_route_type_1_combined,std_avg_time_route_type_1_combined,percentile25_avg_time_route_type_1_combined,percentile75_avg_time_route_type_1_combined,max_avg_time_route_type_2_combined,sum_avg_time_route_type_2_divided_num_flight_combined,median_avg_time_route_type_2_combined,std_avg_time_route_type_2_combined,percentile25_avg_time_route_type_2_combined,percentile75_avg_time_route_type_2_combined,max_avg_time_route_type_3_combined,sum_avg_time_route_type_3_divided_num_flight_combined,median_avg_time_route_type_3_combined,std_avg_time_route_type_3_combined,percentile25_avg_time_route_type_3_combined,percentile75_avg_time_route_type_3_combined,max_complexity_type_None,sum_complexity_type_None,median_complexity_type_None,std_complexity_type_None,percentile25_complexity_type_None,percentile75_complexity_type_None,max_complexity_type_1,sum_complexity_type_1,median_complexity_type_1,std_complexity_type_1,percentile25_complexity_type_1,percentile75_complexity_type_1,max_complexity_type_2,sum_complexity_type_2,median_complexity_type_2,std_complexity_type_2,percentile25_complexity_type_2,percentile75_complexity_type_2,max_complexity_type_3,sum_complexity_type_3,median_complexity_type_3,std_complexity_type_3,percentile25_complexity_type_3,percentile75_complexity_type_3,max_harmonic_complexity_type_None,sum_harmonic_complexity_type_None,median_harmonic_complexity_type_None,std_harmonic_complexity_type_None,percentile25_harmonic_complexity_type_None,percentile75_harmonic_complexity_type_None,max_harmonic_complexity_type_1,sum_harmonic_complexity_type_1,median_harmonic_complexity_type_1,std_harmonic_complexity_type_1,percentile25_harmonic_complexity_type_1,percentile75_harmonic_complexity_type_1,max_harmonic_complexity_type_2,sum_harmonic_complexity_type_2,median_harmonic_complexity_type_2,std_harmonic_complexity_type_2,percentile25_harmonic_complexity_type_2,percentile75_harmonic_complexity_type_2,max_harmonic_complexity_type_3,sum_harmonic_complexity_type_3,median

## Save dataset <a id="save"></a>

In [17]:
path_storage = f"{SECTORS_DATASET_1}"
df_all_sectors.to_csv(path_storage, sep=";", index=False)